In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import datetime
from io import StringIO

from airflow.decorators import dag, task
from airflow.operators.python import get_current_context
from airflow.models import Variable

In [3]:
vgsales = 'vgsales.csv'

In [6]:
year = 1994 + hash(f'{"a-merkulova-22"}') % 23
year

2001

In [28]:
vgsales = pd.read_csv('vgsales.csv')

In [12]:
vgsales.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [6]:
vgsales.query('Year == @year')

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
15,16,Kinect Adventures!,X360,2010.0,Misc,Microsoft Game Studios,14.97,4.94,0.24,1.67,21.82
26,27,Pokemon Black/Pokemon White,DS,2010.0,Role-Playing,Nintendo,5.57,3.28,5.65,0.82,15.32
31,32,Call of Duty: Black Ops,X360,2010.0,Shooter,Activision,9.67,3.73,0.11,1.13,14.64
40,41,Call of Duty: Black Ops,PS3,2010.0,Shooter,Activision,5.98,4.44,0.48,1.83,12.73
54,55,Gran Turismo 5,PS3,2010.0,Racing,Sony Computer Entertainment,2.96,4.88,0.81,2.12,10.77
...,...,...,...,...,...,...,...,...,...,...,...
16437,16440,The Cages: Pro Style Batting Practice,Wii,2010.0,Sports,Konami Digital Entertainment,0.01,0.00,0.00,0.00,0.01
16478,16481,Ishin no Arashi: Shippuu Ryuumeden,DS,2010.0,Strategy,Tecmo Koei,0.00,0.00,0.01,0.00,0.01
16487,16490,Slotter Mania P: Tetsuya Shinjuku vs Ueno,PSP,2010.0,Misc,Dorart,0.00,0.00,0.01,0.00,0.01
16501,16504,"Transformers: War for Cybertron (XBox 360, PS3...",PC,2010.0,Shooter,Activision,0.01,0.00,0.00,0.00,0.01


In [39]:
# Какая игра была самой продаваемой в этом году во всем мире?
vgsales.query('Year == @year') \
       .groupby('Name') \
       .agg({'Global_Sales': 'sum'}) \
       .Global_Sales \
       .idxmax()

'Call of Duty: Black Ops'

In [20]:
# Игры какого жанра были самыми продаваемыми в Европе? Перечислить все, если их несколько
vgsales.query('Year == @year') \
       .groupby('Genre') \
       .agg({'EU_Sales': 'sum'}) \
       .EU_Sales \
       .idxmax()

'Action'

In [34]:
# На какой платформе было больше всего игр, которые продались более чем миллионным тиражом в Северной Америке?
# Перечислить все, если их несколько?
vgsales.query('NA_Sales > 1') \
       .groupby('Platform') \
       .agg({'Name': 'count'}) \
       .Name \
       .idxmax()

'X360'

In [72]:
# У какого издателя самые высокие средние продажи в Японии?
# Перечислить все, если их несколько
vgsales.query('Year == @year') \
       .groupby('Publisher') \
       .agg({'JP_Sales': 'mean'}) \
       .JP_Sales \
       .idxmax()

'Level 5'

In [43]:
# Сколько игр продались лучше в Европе, чем в Японии?

In [68]:
vgsales.query('Year == @year') \
       .query('EU_Sales > JP_Sales') \
       .shape[0]

665

In [7]:
games = pd.read_csv(vgsales).query('Year == @year')

In [62]:
games.groupby('Name').agg({'Global_Sales': 'sum'}).Global_Sales.idxmax()

'Gran Turismo 3: A-Spec'

In [44]:
games.groupby('Name').sum()

,Rank,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Name,,,,,,,
18 Wheeler: American Pro Trucker,4832,2001.0,0.20,0.15,0.00,0.05,0.40
4x4 EVO 2,6903,2001.0,0.18,0.05,0.00,0.01,0.24
4x4 Evolution,5681,2001.0,0.16,0.12,0.00,0.04,0.32
A Ressha de Gyoukou 2001,14869,2001.0,0.00,0.00,0.03,0.00,0.03
ATV Offroad Fury,642,2001.0,2.07,0.26,0.00,0.08,2.41
...,...,...,...,...,...,...,...
Yu-Gi-Oh! The Eternal Duelist Soul (JP sales),4758,2001.0,0.00,0.00,0.40,0.01,0.41
Zoboomafoo: Leapin' Lemurs!,14465,2001.0,0.02,0.01,0.00,0.00,0.03
Zone of the Enders,2746,2001.0,0.30,0.24,0.13,0.08,0.75


In [15]:
games.Name.iloc[0]

'Gran Turismo 3: A-Spec'

In [70]:
games.groupby('Name', as_index = False).agg({'Global_Sales': 'sum'}).Name.idxmax()

TypeError: reduction operation 'argmax' not allowed for this dtype

In [32]:
v_best_EU = games.groupby('Genre').agg({'EU_Sales': 'sum'}).EU_Sales.idxmax()
best_EU = games.query('Genre == @v_best_EU').groupby('Genre').agg({'EU_Sales': 'sum'})
best_EU

,EU_Sales
Genre,
Action,19.25


In [33]:
v_best_EU

'Action'

In [65]:
v_best_platform = games.query('NA_Sales > 1').groupby('Platform').agg({'Name': 'count'}).Name.idxmax()
best_platform = games.query('Platform == @v_best_platform').groupby('Platform').agg({'Name': 'count'})
best_platform.to_csv

<bound method NDFrame.to_csv of           Name
Platform      
PS2        185>

In [35]:
best_platform

,Name
Platform,


In [36]:
v_best_platform

'PS2'

In [74]:
games.groupby('Publisher').agg('JP_Sales').mean().idxmax().to_list()[0]


AttributeError: 'str' object has no attribute 'to_list'

In [47]:
df.query("Year == @y").groupby('Name').sum().Global_Sales.idxmax()[0]

NameError: name 'df' is not defined

In [63]:
data_long = top_data_df.set_index('domain')[['number']].idxmax()[0]

NameError: name 'top_data_df' is not defined

In [45]:
best_EU_JP = games.query('EU_Sales > JP_Sales').shape[0]

In [46]:
best_EU_JP

388

In [66]:
games.sort_values('Global_Sales', ascending=False) \
                       .iloc[0]['Name']

'Gran Turismo 3: A-Spec'